#Week Ten - Assignment: Document Classification

## Prompt:

It can be useful to be able to classify new "test" documents using already classified "training" documents.  A common example is using a corpus of labeled spam and ham (non-spam) e-mails to predict whether or not a new document is spam.  Here is one example of such data:  UCI Machine Learning Repository: Spambase Data Set

For this project, you can either use the above dataset to predict the class of new documents (either withheld from the training dataset or from another source such as your own spam folder).

For more adventurous students, you are welcome (encouraged!) to come up a different set of documents (including scraped web pages!?) that have already been classified (e.g. tagged), then analyze these documents to predict how new documents should be classified.

# Import Packages

In [2]:
import nltk
import random
import pandas as pd
random.seed(1)
import requests
from bs4 import BeautifulSoup
import os

# Loading Dataset

In [5]:
url = "https://spamassassin.apache.org/old/publiccorpus/"
response = requests.get(url)

if response.status_code == 200: #check request
    soup = BeautifulSoup(response.content, 'html.parser')

    links = soup.find_all('a')
    filenames = ["20021010_easy_ham.tar.bz2", "20030228_spam.tar.bz2"]

    save_dir = "./downloaded_emails"
    os.makedirs(save_dir, exist_ok=True)

    for link in links:
        href = link.get('href')
        if href in filenames:
            download_url = url + href

            os.system(f"wget -P {save_dir} {download_url}") # wget downloads files

            print(f"Downloaded {href} successfully.")
else:
    print("Failed to retrieve the webpage.")


Downloaded 20021010_easy_ham.tar.bz2 successfully.
Downloaded 20030228_spam.tar.bz2 successfully.


I first started by extracting data from the spamassassin public corpus which can be found here: https://spamassassin.apache.org/old/publiccorpus/. I downloaded one ham ("20021010_easy_ham.tar.bz2") and one spam ("20030228_spam.tar.bz2") folder.

# Presentation Link